In [ ]:
# ===============================================================
# 03_kerr_flux_sim.ipynb   — Kerr-enhanced quaternionic flux
# ===============================================================
# Colab 〈Cell 1〉：安裝與常數 -----------------------------------
!pip install --quiet scipy

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from math import sqrt, pi

# ----- 物理常數（SI） -------------------------------------------
G     = 6.67430e-11          # m³ kg⁻¹ s⁻²
c     = 2.99792458e8         # m s⁻¹
M_sun = 1.98847e30           # kg
M_pl  = 2.176434e-8          # kg  (Planck mass)

# ----- 黑洞參數：以 Sgr A* 為例 ----------------------------------
M_bh   = 4.0e6 * M_sun       # 4×10⁶ M☉
a_star = 0.5                 # 無因次自旋 (0≤a*<1)

r_g_m  = 2*G*M_bh/c**2       # Schwarzschild half-diam. (m)
print(f"r_g = {r_g_m/1e3:.2e} km,  a* = {a_star}")

# ----- 無因次化 --------------------------------------------------
#   r̂ = r / r_g  ⇒  r_g → 1
a_hat  = a_star
Mratio = M_bh / M_pl         # ≈ 3.6×10⁴⁴

# Δ( r̂ ) 及其導數
Δ  = lambda x: x**2 - x + a_hat**2
dΔ = lambda x: 2*x - 1

# ----- curvature-driven coupling λ̃( r̂ ) -------------------------
kappa   = 2.0e-2
λ_tilde = lambda x: 2*kappa*Mratio / x**3    # 論文 Eq.(E.4)

# ----- ODE：  y = [ψ, χ = dψ/dx] --------------------------------
def rhs(x, y):
    ψ, χ = y
    src  = x**2 * λ_tilde(x) * ψ*(ψ**2 - 1)
    dxd  = (2*x*Δ(x) + x**2*dΔ(x))
    χp   = (src - dxd*χ) / (x**2 * Δ(x))
    return [χ, χp]

# 積分設定：從遠端  r̂=100  往  r̂_+  積分 -------------------------
x_end   = 100.0
x_plus  = (1 + sqrt(1 - a_hat**2)) / 2
x_start = x_plus * 1.02                     # 避開視界
x_eval  = np.geomspace(x_end, x_start, 600) # 反向給點

# 邊界條件 (遠端)：ψ→0,  ψ'→0  ------------------------------------
sol = solve_ivp(rhs,
                t_span=(x_end, x_start),    # 往內積分
                y0=[0.0, 0.0],
                t_eval=x_eval,
                method="Radau", rtol=1e-7, atol=1e-10)

if not sol.success:
    raise RuntimeError(sol.message)

# 反轉回遞增 r̂，以方便閱讀
x_arr  = sol.t[::-1]
ψ_arr  = sol.y[0][::-1]
ε_arr  = 2*ψ_arr

# ----- 理論 ansatz：論文 Eq.(E.5) -------------------------------
r_s0_hat = (8.5e3 * 3.085677e16) / r_g_m    # 8.5 kpc → r̂
boost    = 1.0 / (1 - a_hat / x_arr)
ε_ansatz = 2*np.tanh( x_arr / (r_s0_hat * boost) )

# --------------------- 圖形 --------------------------------------
plt.figure(figsize=(7,5))
plt.loglog(x_arr, ε_arr,    lw=2, label="Numerical ε(r)")
plt.loglog(x_arr, ε_ansatz, "--", lw=2, label="Analytic ansatz")
plt.axvline(x_plus, ls=":", c='k', alpha=.5); plt.text(0.93, 0.3, "$r_+$",
    transform=plt.gca().transAxes)
plt.xlabel(r"$\hat r = r/r_g$")
plt.ylabel(r"$\epsilon(r)$")
plt.title("Quaternionic flux in Kerr spacetime (fixed)")
plt.legend(); plt.grid(ls="--", alpha=0.3)
plt.tight_layout(); plt.show()

# ----- 估算 δc_g / c  (LISA, f = 0.01 Hz) ------------------------
f_LISA = 1e-2                       # Hz
k_SI   = 2*pi*f_LISA / c            # s⁻¹
k_hat  = k_SI * r_g_m / c           # 換成無因次 (自然單位)
idx10  = np.argmin(np.abs(x_arr - 10))
ψ10    = ψ_arr[idx10]
deltac = (ψ10**2 * (a_hat/x_arr[idx10])**2) / (2 * k_hat**2)

print(f"\n在 10 r_g： ε ≃ {ε_arr[idx10]:.3e}")
print(f"預測 δc_g/c ≃ {deltac:.2e}")
